In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

In [3]:
sql_salaries = '''
drop table if exists salaries;
create table salaries (emp_id integer, depname varchar(20), emp varchar(20), role varchar(20), salary integer);
insert into salaries (emp_id, depname, emp,role, salary) values (1, 'dep1','Christiano','Manager', 1000);
insert into salaries (emp_id, depname, emp,role, salary) values (2, 'dep1','Eric','Manager', 1500);
insert into salaries (emp_id, depname, emp,role, salary) values (3, 'dep1','Ryan','Director', 2000);
insert into salaries (emp_id, depname, emp,role, salary) values (4, 'dep1', 'Bryan','Director',1500);
insert into salaries (emp_id, depname, emp,role, salary) values (5, 'dep2', 'Doreen','Manager',2000);
insert into salaries (emp_id, depname, emp,role, salary) values (6, 'dep2', 'Gabriel','Manager',3000);
insert into salaries (emp_id, depname, emp,role, salary) values (7, 'dep2', 'Angel','Manager',2400);
insert into salaries (emp_id, depname, emp,role, salary) values (8, 'dep2', 'Robin','Director',2700);
insert into salaries (emp_id, depname, emp,role, salary) values (9, 'dep3','Sarah','Manager', 1900);
insert into salaries (emp_id, depname, emp,role, salary) values (10, 'dep3', 'Peter','Manager',1700);
'''

In [4]:
sql_salaries = '''
insert into salaries (emp_id, depname, emp,role, salary) values (9, 'dep3','Sarah','Manager', 1900);
insert into salaries (emp_id, depname, emp,role, salary) values (10, 'dep3', 'Peter','Manager',1700);
'''

In [4]:
cur.executescript(sql_salaries)
con.commit()

In [5]:
pd.read_sql(
    """
SELECT *
FROM salaries
    """,
    con,
)

,emp_id,depname,emp,role,salary
0,1,dep1,Christiano,Manager,1000
1,2,dep1,Eric,Manager,1500
2,3,dep1,Ryan,Director,2000
3,4,dep1,Bryan,Director,1500
4,5,dep2,Doreen,Manager,2000
5,6,dep2,Gabriel,Manager,3000
6,7,dep2,Angel,Manager,2400
7,8,dep2,Robin,Director,2700
8,9,dep3,Sarah,Manager,1900
9,10,dep3,Peter,Manager,1700


In [6]:
pd.read_sql(
    """
SELECT avg(salary) as 'avg'
FROM salaries;
    """,
    con,
)

,avg
0,1970.0


In [7]:
pd.read_sql(
    """
SELECT emp_id,emp, depname, salary, avg(salary) OVER() as 'avg'
FROM salaries;
    """,
    con,
)

DatabaseError: Execution failed on sql '
SELECT emp_id,emp, depname, salary, avg(salary) OVER () as 'avg'
FROM salaries;
    ': near "(": syntax error

In [11]:
pd.read_sql(
    """
SELECT emp_id,emp, depname, salary, abs(salary - avg(salary) OVER ()) as 'diff_with_avg'
FROM salaries;
    """,
    con,
)

DatabaseError: Execution failed on sql '
SELECT emp_id,emp, depname, salary, abs(salary - avg(salary) OVER ()) as 'diff_with_avg'
FROM salaries;
    ': near "OVER": syntax error

In [ ]:
pd.read_sql(
    """
SELECT emp_id,emp, depname, salary, avg(salary) OVER (PARTITION BY depname) as 'avg_by_dept'
FROM salaries;
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
SELECT emp_id,emp, depname, salary, abs(salary - avg(salary) OVER (PARTITION BY depname))  as 'diff_with_avg_by_dept'
FROM salaries;
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
SELECT emp_id,emp, depname, salary, sum(salary) OVER () as 'sum'
FROM salaries;
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
SELECT emp_id,emp, depname, salary, salary*100/(sum(salary) OVER ()) as 'prc'
FROM salaries;
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
SELECT emp_id,emp, depname, salary, round(salary*100/(sum(salary) OVER ()),1) as 'prc'
FROM salaries;
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
SELECT emp_id,emp, depname, salary, cast(salary*100/(sum(salary) OVER ()) as float) as 'prc'
FROM salaries;
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
SELECT emp_id,emp, depname, salary, round(cast(salary as float)*100/(sum(salary) OVER ()),2) as 'prc'
FROM salaries;
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
SELECT emp_id,emp, depname, salary, round(salary*100/cast(sum(salary) OVER () as float),2) as 'prc'
FROM salaries;
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
SELECT emp_id,emp, depname, salary, round(salary*100.0/(sum(salary) OVER ()),2) as 'prc'
FROM salaries;
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
    
 select q.*, 
 sum(q.prc) OVER () as 'new_prc' 
 
 from 
 (
SELECT emp_id,
emp, --'сотрудник'
depname, 
salary, 
round(salary*1.0/(sum(salary) OVER ()),3) as 'prc'

FROM salaries) q

    """,
    con,
)

In [ ]:
pd.read_sql(
    """
SELECT 1/2
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
SELECT 1.0/2
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
SELECT 1/2.0
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
SELECT emp_id,emp, depname, salary, sum(salary) OVER (PARTITION BY depname) as 'sum'
FROM salaries;
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
SELECT emp_id,emp, depname, salary, sum(salary) OVER (PARTITION BY depname ORDER BY salary) as 'sum'
FROM salaries;
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
SELECT emp_id,emp, depname, salary,
sum(salary) OVER (PARTITION BY depname ORDER BY salary ROWS BETWEEN CURRENT ROW AND 1 FOLLOWING) as 'sum'
FROM salaries;
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
SELECT emp_id, emp,depname, salary, row_number() OVER (PARTITION BY depname) row_num_by_dept
FROM salaries;
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
SELECT emp_id, emp,depname, salary, row_number() OVER (PARTITION BY depname ORDER BY salary DESC) row_num_by_dept
FROM salaries
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
SELECT emp_id, emp,depname, salary, rank() OVER (PARTITION BY depname ORDER BY salary DESC) rank_by_dept
FROM salaries
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
SELECT emp_id, emp,depname, salary, dense_rank() OVER (PARTITION BY depname ORDER BY salary DESC) rank_by_dept
FROM salaries
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
select * from (
SELECT emp_id, emp,depname, salary, row_number() OVER (PARTITION BY depname ORDER BY salary DESC) row_num_by_dept
FROM salaries
) r where r.row_num_by_dept = 1
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
with ranks as (
SELECT emp_id, emp,depname, salary,
row_number() OVER (PARTITION BY depname ORDER BY salary DESC) row_num_by_dept
FROM salaries
) 
select * from ranks where row_num_by_dept = 1
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
with ranks as (
SELECT emp_id, emp,depname, salary,
row_number() OVER (PARTITION BY depname ORDER BY salary DESC) row_num_by_dept
FROM salaries
) 
select * from ranks where row_num_by_dept = 1
union all
select * from ranks where row_num_by_dept = 2
union all
select * from ranks where row_num_by_dept = 3
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
SELECT emp_id, emp,depname, salary,NTILE(2) OVER(ORDER BY salary DESC) AS 'NTILE' 
FROM salaries
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
SELECT emp_id, emp,depname, salary,LAG(salary) OVER(PARTITION BY depname ORDER BY salary DESC) AS 'LAG' 
FROM salaries
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
SELECT emp_id, emp,depname, salary,LEAD(salary) OVER(PARTITION BY depname ORDER BY salary DESC) AS 'LEAD' 
FROM salaries
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
SELECT emp_id, emp,depname, salary,FIRST_VALUE(salary) OVER(PARTITION BY depname) AS 'FIRST_VALUE' 
FROM salaries
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
SELECT emp_id, emp,depname, salary,LAST_VALUE(salary) OVER(PARTITION BY depname) AS 'LAST_VALUE' 
FROM salaries
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
SELECT emp_id, emp,depname, salary,LAST_VALUE(salary) OVER(PARTITION BY depname ORDER BY salary) AS 'LAST_VALUE' 
FROM salaries
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
SELECT emp_id, emp,depname, salary,LAST_VALUE(salary) OVER(PARTITION BY depname ORDER BY salary
ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
) AS 'LAST_VALUE' 
FROM salaries
    """,
    con,
)

1. Какую часть зарплат от общей суммы получают менеждеры, а какую директора?

In [ ]:
pd.read_sql(
    """

SELECT s.*, sum(salary) over() as 'sum', sum(s.salary) over(partition by s.role) as 'by_role' ,
 (sum(s.salary) over(partition by s.role) )*100.0/(sum(salary) over()) as 'prc_role'
FROM salaries s 

    """,
    con,
)

2. На каком месте по величине зарплаты находятся сотрудники, у которых первая буква в имени "R"? 

In [ ]:
pd.read_sql(
    """
select q.* from (
SELECT emp_id, emp,depname, salary, dense_rank() OVER ( ORDER BY salary DESC) rank_by_dept
FROM salaries
) q
where emp like 'R%'
    """,
    con,
)

3. Выведите процент менеджеров и директоров по каждому из департаментов.

In [ ]:
pd.read_sql(
    """
    select q.depname,q.role,q.prc_man_dir from (
SELECT depname,role,salary,
--count() over(partition by depname,role ) as 'cnt_dep_role',count() over(partition by depname ) as 'cnt_dep',
(count() over(partition by depname,role ))*100.0 /count() over(partition by depname ) as 'prc_man_dir'
FROM salaries
) q
group by q.depname,q.role,q.prc_man_dir
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
SELECT depname,role
FROM salaries q
group by q.depname,q.role
    """,
    con,
)

4. Выведите среднюю зарплату по 1 департаменту, а также по 2 и 3 совместно.

In [ ]:
pd.read_sql(
    """
SELECT emp_id,depname,emp,role,salary,
avg(salary) over (partition by depname)
FROM salaries
where depname = 'dep1'
union all
SELECT emp_id,depname,emp,role,salary
,avg(salary) over (partition by depname)
FROM salaries
where depname != 'dep1'
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
    with  avg_sal as (
SELECT distinct depname,
avg(salary) over (partition by depname) as 'avg_'
FROM salaries
)
select * from avg_sal where depname = 'dep1'
union all
select 'dep2+dep3', sum(avg_) from avg_sal where depname != 'dep1'

    """,
    con,
)

5. Какой процент от суммы зарплат по всех организации получают 10% сотрудников с самыми большими зарплатами? 20%?30%?

In [ ]:
pd.read_sql(
    """
  with table1 as (  
SELECT emp_id,depname,emp,role,salary,
sum(salary) over () as 'sum_all',
count() over () as 'cnt_all',
count() over (order by salary desc ROWS BETWEEN UNBOUNDED PRECEDING AND current row) as 'sum_emp',
sum(salary) over (order by salary desc ROWS BETWEEN UNBOUNDED PRECEDING AND current row) as 'sum_sal'
FROM salaries
)

select sum_emp*100.0/cnt_all as 'true_prc',
table1.salary,table1.sum_all,table1.sum_sal,
sum_sal*100.0/sum_all


from table1
    """,
    con,
)

In [ ]:
pd.read_sql(
    """
  with table1 as (  
SELECT emp_id,depname,emp,role,salary,
sum(salary) over () as 'sum_all',
--count() over () as 'cnt_all',
count() over (order by salary desc ROWS BETWEEN UNBOUNDED PRECEDING AND current row) as 'sum_emp',
NTILE(2) OVER(order by salary)
FROM salaries
)

select *  from table1
    """,
    con,
)